Trying to include other two camera images.

## Import Python Modules:

In [1]:
import csv
import cv2
import numpy as np
import time

from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D 

Using TensorFlow backend.


## Import Data:

In [2]:
# List for each line of the csv file
lines = []

# Open *.csv file using csv module
with open('C:\\Users\\vDrone\\Desktop\\Project 3. Behaviour Cloning\\data\\driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)   
    for line in reader:
        lines.append(line)

        
# Lists for images and steering value
images = []
measurements = []

# Load images and steering values and append them to the lists
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]

        current_path = 'C:\\Users\\vDrone\\Desktop\\Project 3. Behaviour Cloning\\data\\IMG\\' + filename
        current_path = str(current_path)

        image = cv2.imread(current_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # For drive.py and OpenCV conflict
        images.append(image)
    #     print(images)
        
        correction = 0.1
        measurement = float(line[3])
        
        if i == 0:
            measurements.append(measurement)        
        elif i == 1:
            measurements.append(measurement + correction)
        elif i == 2:
            measurements.append(measurement - correction)
     
# print the length of lists to verify whether data is loaded properly or not.    
print(len(images))
print(len(measurements))

24108
24108


In [3]:
print(measurements[0:6])

[0.0, 0.1, -0.1, 0.0, 0.1, -0.1]


## Data Augumention :

In [4]:
augmented_images, augmented_measurements = [], []

for image,measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)

## Convert Data to Numpy Array:

In [5]:
# Convert Images and Steering values to numpy arrays since keras requires them in that form
X_train = np.array(augmented_images)
Y_train = np.array(augmented_measurements)

print(len(X_train), len(Y_train))

48216 48216


## Simplest Model for checking:

In [6]:
model = Sequential()

# First Layer: Normalizing, Mean Centering the image and cropping the image
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3))) 
model.add(Cropping2D(cropping=((70,25),(0,0))))

# Second Layer: Convolutional Layer
model.add(Convolution2D(24,5,5, subsample=(2,2), activation="relu"))

# Third Layer: Convolutional Layer
model.add(Convolution2D(36,5,5, subsample=(2,2), activation="relu"))

# Fourth Layer: Convolutional Layer
model.add(Convolution2D(48,5,5, subsample=(2,2), activation="relu"))

# Fifth Layer: Convolutional Layer
model.add(Convolution2D(64,3,3,activation="relu"))

# Sixth Layer: Convolutional Layer
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Flatten())

# Seventh Layer: Fully connected Layer
model.add(Dense(100))

# Eightth Layer: Fully connected Layer
model.add(Dense(50))

# Nineth Layer: Fully connected Layer
model.add(Dense(10))

# Output Layer
model.add(Dense(1))

t1 = time.time()
print('Training.....')

# Compile and train
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

t2 = time.time()
print('Time required to train the model: ', t2-t1)

# save trained model
model.save('model.h5')
print('Model saved to the directory.')

Training.....
Train on 38572 samples, validate on 9644 samples
Epoch 1/5
38572/38572 [==============================] - 478s - loss: 0.0113 - val_loss: 0.0118
Epoch 2/5
38572/38572 [==============================] - 482s - loss: 0.0098 - val_loss: 0.0113
Epoch 3/5
38572/38572 [==============================] - 480s - loss: 0.0093 - val_loss: 0.0111
Epoch 4/5
38572/38572 [==============================] - 1458s - loss: 0.0089 - val_loss: 0.0111
Epoch 5/5
38572/38572 [==============================] - 482s - loss: 0.0085 - val_loss: 0.0116
Time required to train the model:  3384.6756026744843
Model saved to the directory.
